In [3]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

import openmm as mm
import openmm.app as app
import openmm.unit as unit
from mdtraj.reporters import HDF5Reporter
import copy

import random

# # Path: pymdna/__init__.py, prototype of the package and atomic is not properly referenced in the package at genertors.py now I just explicitly define the path loction
# import pymdna as mdna
import sys
sys.path.append('/Users/thor/surfdrive/Projects/pymdna/')
import pymdna as mdna 
%load_ext autoreload
%autoreload 2

joblib is not installed. Falling back to sequential computation.


In [2]:
# Make DNA from sequence
sequence = 'CATCATCAT'
pdb = mdna.sequence_to_pdb(sequence=sequence,output='GROMACS')
pdb.save('./pdbs/wild.pdb')

Sequence: CAT
Number of base pairs: 3


In [3]:
# Mutate DNA 
mutations = {0: 'A', 1: 'T', 2: 'G'}
mutant = mdna.Mutate(pdb, mutations,complementary=True)
mutant_pdb = mutant.get_traj()
mutant_pdb.save('./pdbs/mutant.pdb')

mutant_sequence = mdna.get_sequence_letters(mutant_pdb)
print('New sequence:',''.join(mutant_sequence))

New sequence: ATG


In [4]:
# Mutation (non-canonical//fluorescent base analogue)
# P = 2AP (2-aminopurine)  https://doi.org/10.1002/anie.201001312 (2kv0)
# D = tricyclic cytosin base analogue (1tuq)
mutations = {1: 'P', 2 : 'D'}
mutant = mdna.Mutate(pdb, mutations,complementary=True)
new_pdb = mutant.get_traj()
new_pdb.save('./pdbs/baseanalogues.pdb')

mutant_sequence = mdna.get_sequence_letters(new_pdb)
print('New sequence:',''.join(mutant_sequence))


New sequence: CPD


In [5]:
# Methylate DNA
methylator = mdna.Methylate(pdb, methylations=[0])
pdb_meth = methylator.get_traj()
pdb_meth.save('./pdbs/methylated.pdb')

meth_sequence = mdna.get_sequence_letters(pdb_meth)
print('New sequence:',''.join(meth_sequence))

New sequence: CCA


In [30]:
# Methylate DNA
sequence = 'GCGCTATGCGCGGCGCGCTCGCTCACGACTACGATCAGCTACG'
long_pdb = mdna.sequence_to_pdb(sequence=sequence,output='GROMACS')
methylator = mdna.Methylate(long_pdb, CpG=True,leading_strand=0)
pdb_meth_long = methylator.get_traj()

pdb_meth_long.save('./pdbs/methylated.pdb')
meth_sequence = mdna.get_sequence_letters(pdb_meth_long)
print('New sequence:',''.join(meth_sequence))

Sequence: GCGCTATGCGCGGCGCGCTCGCTCACGACTACGATCAGCTACG
Number of base pairs: 43
Methylate all C in CpG context, superseeds methylations list.
Methtylating: [1, 8, 10, 13, 15, 19, 25, 31, 41]
New sequence: GCGCTATGCGCGGCGCGCTCGCTCACGACTACGATCAGCTACG


In [6]:
# Hoogsteen flip
flipper = mdna.Hoogsteen(pdb,fliplist=[1],deg=180,verbose=True)
pdb_flipped = flipper.get_traj()
pdb_flipped.save('./pdbs/WC_to_HG.pdb')


Flipped residues [1] by 3.141592653589793 radians


In [7]:
def point_mutation(sequence, position=None, new_nucleotide=None):
    if position is None:
        position = random.randint(0, len(sequence) - 1)
    if new_nucleotide is None:
        nucleotides = ['A', 'T', 'C', 'G']
        new_nucleotide = random.choice([n for n in nucleotides if n != sequence[position]])
    
    mutated_sequence = list(sequence)
    mutated_sequence[position] = new_nucleotide
    return ''.join(mutated_sequence)

def radiate_system(traj, new_sequence, complementary=True, chainids=[0,1], verbose=False):

    pdb = copy.deepcopy(traj)   
    if verbose:
        print('--- current stat of the system ---')
    s = mdna.get_sequence_letters(pdb)
    if len(s) != len(new_sequence):
        raise ValueError('The length of the new sequence does not match the length of the current sequence')
    if verbose:
        print(len(s),s)
        print(''.join(s))
    mutations = mdna.get_mutations(s,new_sequence)
    if verbose:
        print(f'start mutation ---- {mutations} ----')    
    dna = pdb.atom_slice(pdb.top.select(f'chainid {chainids[0]} {chainids[1]}'))
    if verbose:
        for c in dna.top.chains:
            print(c.index, c._residues)

    mutant = mdna.Mutate(dna, mutations,complementary=complementary)
    mutant_sequence = mdna.get_sequence_letters(mutant.mutant_traj)
    if verbose:
        print(mutant_sequence)
        print(''.join(mutant_sequence))
    new_traj = mutant.get_traj()
    if verbose:
        for c in new_traj.top.chains:
            print(c.index, c._residues)
        print('--- end radiation ---')
    return new_traj

# Example usage
sequence = mdna.get_sequence_letters(pdb)
point_mutations = np.unique([point_mutation(sequence) for _ in range(100)])

for i, new_sequence in enumerate(point_mutations):
    mutant = radiate_system(pdb, list(new_sequence))
    print(new_sequence,f'saved as point_mutant_{i}.pdb')
    mutant.save(f'./pdbs/point_mutant_{i}.pdb')


AAT saved as point_mutant_0.pdb
CAA saved as point_mutant_1.pdb
CAC saved as point_mutant_2.pdb
CAG saved as point_mutant_3.pdb
CCT saved as point_mutant_4.pdb
CGT saved as point_mutant_5.pdb
CTT saved as point_mutant_6.pdb
GAT saved as point_mutant_7.pdb
TAT saved as point_mutant_8.pdb
